<html>
    <h1 style="color:red;font-family:Times New Roman;text-align:center;text-decoration:underline;text-decoration-color:'blue';">Libraries used :</h1>
    <ul style="list-style-type: none;">
        <li>
             <img width=150px height=150px  src="imgs/1200px-Sympy_logo.svg.png"  alt="Sympy">
        </li>
        <br>
        <li>
             <img width=300px height=300px  src="imgs/logo2.webp" alt="Matplotlib">
        </li>
        <br>
        <li>
             <img width=200px height=200px src="imgs/1200px-NumPy_logo.svg.png" alt="Numpy">
        </li>
        <br>
        <li>        
            <img width=200px height=200px  src="imgs/scipy.png" class="rounded" alt="Scipy">
        </li>
        <br>
        <li>
             <img width=400px height=400px  src="imgs/pandas_logo.png" class="rounded" alt="Pandas">
        </li>
        <br>
        <li>
             <img width=150px height=150px  src="imgs/Plotly-logo-01-square.png" class="rounded" 
alt="Pandas">
        </li>
        </ul>
    

In [5]:
import numpy as np
from numba import jit, int32, float32, prange
from scipy.sparse import diags
from scipy.integrate import RK45, solve_ivp
import scipy.sparse as sp
import matplotlib.pyplot as plt
import matplotlib
import requests
from time import time
import pandas as pd
import plotly.express as px
import psutil
import time
import os
import math
#import pandas as pd
from tqdm import tqdm_notebook

from IPython.core.display import display, HTML
from IPython.display import IFrame
display(HTML("<style>.container { width:100% !important; }</style>"))

\begin{equation}
\underline {System \ of \ equations :}
\\\ u_t + u  u_x + v_x + \beta u_{xx} = 0 
\\ v_t + (uv)_x - \beta v_{xx} = 0 
\end{equation}

\begin{equation}
\underline {  Galerkin \ Solution :}
\\ u^h = 
\sum_{i= 0}^{n} a_i(t)  \phi_i(x) 
\end{equation}

\begin{equation}
\underline {System \ of \ equations :}
\\\ \int_0^1 { (w u_t + wu  u_x + wv_x + \beta w u_{xx})} \ dx= 0 
\\ \int_0^1 { (wv_t + w (uv)_x - \beta w v_{xx})} \ dx= 0 
\end{equation}

<h3 align="center" style="color:#FFE647;">Equation  (i) </h3> 
\begin{equation}
\\\ \int_0^1  (w u_t  + wuu_x + wv_x ) \ dx   - \beta \int_0^1 {w_x u_x} \ dx  +\beta \ (w(1) u_x(1) - w(0) u_x(0)) = 0 
\\  (A \alpha^{'} + \alpha^TZ\alpha +C\gamma)\hspace{5mm} - \beta D  \alpha  +  \beta ( E \alpha) \hspace{10mm}  = 0 
\\ w = {\phi_j}
\end{equation}

<h3 align="center" style="color:#FF3647;">Equation  (ii) </h3> 

\begin{equation}
\\\ \int_0^1  (w v_t  + w(uv)_x ) \ dx + \beta \int_0^1 {w_x v_x} \ dx - \beta \ (w(1) v_x(1) - w(0) v_x(0)) = 0 
\\  (A \gamma^{'} + \alpha^T(2Z)\gamma)\hspace{5mm} + \beta D  \gamma -\beta (  E \gamma) \hspace{10mm} = 0 
\\ w = {\phi_j}
\end{equation}

<h3 align="center" style="color:#FF3647;">Initial Conditions </h3><br> 
\begin{equation}
\\ u(x,0)=λ−2|β|k \ coth(kξ) 
\\  v(x,0)=−2|β|(|β|+β)k^2csch^2(kξ)
\\ ξ=x+x_0
\\ \text{\(\ x_0 \), \(\ k  \), λ  are arbitrary constants }
\\ k=0.1, \ λ=0.005,  \ β=1.5 \ and \ x_0=1
\end{equation}

In [3]:

class Heat:
    
    
    def __init__(self, N=10):
        
        self.N = N
        self.H = 1/N
        self.domain = np.linspace(0, 1, N+1) 
        self.t = 1
        
        #Set initial conditions here     ---> t=0
        self.k = .1
        self.ld = .005
        self.beta = 1.5
        self.x0 = 1
        self.B = math.sqrt(self.beta**2)
        self.xi = self.x0 + self.domain
        self.alpha = self.ld - 2 * self.B*self.k/np.tanh(self.k * self.xi)
        self.gamma = -2 * self.B * (self.B + self.beta) * (self.k/np.sinh(self.k*self.xi))**2 
        
        #Set solution
        self.exact_alpha = lambda t: self.ld - 2 * self.B*self.k/np.tanh(self.k * self.xi - self.ld * t)
        self.exact_gamma = lambda t: -2 * self.B * (self.B + self.beta) * (self.k/np.sinh(self.k*self.xi  - self.ld* t))**2
        
        #Define boundary conditions   !!!!are a function of time
        self.alpha_b_c_start = lambda t: self.ld - 2 * self.B*self.k/np.tanh(self.k * (self.x0 + 0) - self.ld * t)
        self.alpha_b_c_end = lambda t: self.ld - 2 * self.B*self.k/np.tanh(self.k * (self.x0 + 1) - self.ld * t)
        self.gamma_b_c_start = lambda t: -2 * self.B * (self.B + self.beta) * (self.k/np.sinh(self.k*(self.x0 + 0)  - self.ld* t))**2
        self.gamma_b_c_end = lambda t: -2 * self.B * (self.B + self.beta) * (self.k/np.sinh(self.k*(self.x0 + 0)  - self.ld* t))**2
        
        ################Thomas Algorithm Component##################################
        def c_dash_transform(l):
            
            l[0] /= self.A[0]
            for i in range(1, self.N):
                  l[i] = (l[i])/ (self.A[i]-(self.H/6)* l[i-1])
            return l        
        ################ Thomas Algorithm Component ##################################

        ################Create Nonlinear term################  
        def matZ(i):
                if i == 0 :
                    row = np.array([0, 0, 1, 1])
                    col = np.array([0, 1, 0, 1])
                    data = np.array([-1/3, 1/3, -1/6, 1/6])
                    return sp.csr_matrix((data, (row, col)), shape=(N+1, N+1), dtype=np.float32)        
                elif i == self.N:
                    row = np.array([self.N-1, self.N - 1, self.N, self.N])
                    col = np.array([self.N-1, self.N , self.N -1, self.N])
                    data = np.array([-1/6, 1/6, -1/3, 1/3])
                    return sp.csr_matrix((data, (row, col)), shape=(N+1, N+1), dtype=np.float32)   
                else:
                    row = np.array([i-1]*3 + [i]*3 + [i+1]*3)
                    col = np.array([col for col in range(i-1, i+1 + 1)]*3, dtype=np.float32)
                    data = np.array([-1/6, 1/6, 0, -1/3, 0, 1/3, 0, -1/6, 1/6 ])
                    return sp.csr_matrix((data, (row, col)), shape=(N+1, N+1), dtype=np.float32)    
              
        ################ Main diagonal of A ##################################
        self.A = self.H* np.array([1/3] + [2/3]*(N-1) + [1/3]) # 1 -1 diagonals have element h/6 only
        self.c_dash = c_dash_transform(self.H *np.array( [1/6]*(N))) # side diagonals transformed


        ################ Vertical stack of N X N matrices  : a N X N X N tensor ########################
        self.Z = np.array([matZ(i) for i in prange(N+1)])
        
        ##### phi . phi'##################
        self.C = .5  * sp.csr_matrix(diags([      [1 for i in range(N)],  \
                                      [-1] + [0]*(N-1) + [1], \
                                        [-1 for i in range(N)]],  \
                             [1, 0, -1] ), dtype=np.float32)
        
        ################ sparse Matrix ########################
        self.D = N * sp.csr_matrix(diags([      [-1 for i in range(N)],  \
                                      [1] + [2]*(N-1) + [1], \
                                        [-1 for i in range(N)]],  \
                             [1, 0, -1] ), dtype=np.float32)
        
        ########    D - E ########################
        self.D[0, 0] -= N     
        self.D[0, 1] -= -N
        self.D[N, N-1] -= -N
        self.D[N, N] -= N

      
    def error(self):
        return  [np.linalg.norm(self.exact_alpha(1) -  self.approx_alpha, 2),
              np.linalg.norm(self.exact_gamma(1) -  self.approx_gamma, 2)]
              

    

        
    #  y' = func(t, y) 
    def func (self, t,  y):
        
        bc_s = [self.alpha_b_c_start(t), self.alpha_b_c_end(t), self.gamma_b_c_start(t), self.gamma_b_c_end(t)]
        
        def apply_bc():
            d1[0, 0], d1[-1, 0], d2[0, 0], d2[-1, 0] = bc_s
                 
        
        # Thomas Algorithm 
        def thomas_algorithm( d):
            d[0] /= self.A[0]
            for i in range(1, self.N + 1):
                d[i, 0] = (d[i, 0] - (self.H/6)*d[i-1])/(self.A[i]-(self.H/6)*self.c_dash[i-1])
            for i in range(self.N-1, -1, -1): # Last entry is left unchanged ;see that i+1
                d[i, 0] -= self.c_dash[i]*d[i+1]                 
            return d     
        

        d1, d2 = np.split(y, 2, axis =0)
        apply_bc()
        d1 = (  sp.csr_matrix(np.vstack([-d2.T.ravel() * i for i in self.Z]))+ self.beta*self.D).dot(d1) -self.C.dot(d2)
        d2 = (  2*sp.csr_matrix(np.vstack([-d1.T.ravel() * i for i in self.Z]))- self.beta*self.D).dot(d2)
        l = [d1, d2] # only for parallelizing
        d1, d2 = [thomas_algorithm(l[i]) for i in prange(2)]
        apply_bc()
        l = np.concatenate([d1, d2], axis = 0)

        return l


    # LSODA
    def algo_1(self):
        
        self.algo = solve_ivp(self.func,t_span=(0, self.t + .1),  y0=np.concatenate([self.alpha, self.gamma], axis=0), \
                   t_eval= np.array([self.t]), vectorized=True, method="LSODA")
        self.approx_alpha, self.approx_exact =  np.split(self.algo.y, 2, axis =0)
        self.approx_alpha = self.approx_alpha.reshape(-1, )
        self.approx_gamma = self.approx_gamma.reshape(-1, )
        


In [4]:
Ns = [2**r for r in range(2, 4)]
l =[]
for i in tqdm_notebook(Ns):
    heateqn = Heat(i)

    t0 = time.time()

    heateqn.algo_1()
    l.append([i, *heateqn.error()])
    if i!= Ns[-1]:
        del heateqn
    print(f"Time taken for algo to run for {i} basis",  time.time()-t0)

c:\users\dell\desktop\python~1\maths\lib\site-packages\scipy\integrate\_ode.py:1348: UserWarning:

lsoda: Repeated convergence failures (perhaps bad Jacobian or tolerances).



ValueError: need at least one array to concatenate